In [40]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus


In [41]:
# Credenciales base 3
user = "postgres"
password = "admin123"   # tu contraseña real
host = "localhost"
port = "5432"
database = "customers"

In [42]:
# Codificar la contraseña para URL
password_encoded = quote_plus(password)

# Crear motor de conexión
engine = create_engine(
    f"postgresql+psycopg2://{user}:{password_encoded}@{host}:{port}/{database}"
)

# Leer CSV
df = pd.read_csv("Customer_DF.csv")

# Subir a PostgreSQL
df.to_sql("counterfeit_transactions", engine, if_exists="replace", index=False)

print("✅ CSV cargado correctamente en PostgreSQL")

✅ CSV cargado correctamente en PostgreSQL


In [43]:
# Ruta CSV
csv_path = "Customer_DF.csv"

# Cargar CSV en pandas
df = pd.read_csv(csv_path)
print("Dimensiones del dataset:", df.shape)
df.head()

Dimensiones del dataset: (168, 10)


,Unnamed: 0,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,No_Transactions,No_Orders,No_Payments,Fraud
0,0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",2,2,1,False
1,1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,3,3,7,True
2,2,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",5,3,2,False
3,3,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",3,3,1,False
4,4,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",7,7,6,True


In [44]:

df.to_sql("customer_df", engine, if_exists="replace", index=False)
print("✅ Datos cargados en la tabla 'customer_df'")


✅ Datos cargados en la tabla 'customer_df'


# ## 2. DataFrame de Contacto (`df_contacto`)
# Incluye los datos de identificación del cliente: email, teléfono, dispositivo, IP.
# **Filtros:**
# 1. Correos con dominio `gmail.com`.
# 2. Dispositivos móviles.
# 3. IPs duplicadas.

In [50]:
df_contacto = df[["customerEmail","customerPhone","customerDevice","customerIPAddress"]].copy()
print("DataFrame Contacto:", df_contacto.shape)
display(df_contacto.head(10))

# Filtro 1: emails de Gmail
gmail_users = df_contacto[df_contacto["customerEmail"].str.contains("gmail", case=False, na=False)]
display(gmail_users.head())

# Filtro 2: dispositivos móviles
mobile_users = df_contacto[df_contacto["customerDevice"].str.contains("mobile", case=False, na=False)]
display(mobile_users.head())

# Filtro 3: IPs duplicadas
duplicated_ips = df_contacto[df_contacto.duplicated(subset=["customerIPAddress"], keep=False)]
display(duplicated_ips.head())


DataFrame Contacto: (168, 4)


,customerEmail,customerPhone,customerDevice,customerIPAddress
0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40
1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202
2,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0
3,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af
4,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192
5,robinsoncynthia@dunn.com,+65(5)8997489788,hbww39m1wenbq4ymjhsx,43.166.163.164
6,samuel15@saunders-rhodes.com,05045239487,up7afvce5v0omniqlevq,33.67.35.30
7,johnlowery@gmail.com,044-642-9860,rjb0adai34izqvys4938,212.144.68.190
8,jameslandry@rodriguez.com,1-885-168-1310x338,yz4l2bgbvzra5eb5qcan,c899:ff67:6864:4365:def3:3023:655f:5181
9,ubranch@rivera-parker.info,(772)964-8162x6310,6mpx027neyz6sh04n3ue,86.16.65.228


,customerEmail,customerPhone,customerDevice,customerIPAddress
7,johnlowery@gmail.com,044-642-9860,rjb0adai34izqvys4938,212.144.68.190
15,gonzalesjackson@gmail.com,1-410-373-1228x129,hmkn4nmfc5q5mgify2zw,229.2.225.188
26,avaldez@gmail.com,203-573-2041,bwgvb5w8dm6upvi7ucqc,46.130.37.206
29,catherine64@gmail.com,563-396-2038x288,myjxxvd3jfn1eymfy50n,1289:d2a9:c65d:8fc7:7be4:129b:d7b:cf7b
40,johnlowery@gmail.com,737-377-9701x577,jz66ap43id2y35ivfqa5,6c21:ac1d:2089:68fa:abb7:8c00:525f:6588


,customerEmail,customerPhone,customerDevice,customerIPAddress


,customerEmail,customerPhone,customerDevice,customerIPAddress
46,jamescampbell@randall-pacheco.biz,599-142-6783,eaul63h9guyvwuymfn3z,45.203.99.249
80,warrenedward@arnold.com,201.737.5119x64000,6py7ewqjkxjw8zjl9eri,45.203.99.249
88,shelby24@hotmail.com,(071)934-3315x27613,obb41ux0zfa2td5ajo9s,45.203.99.249
141,christinemills@mcgee.com,(651)355-4709,zbn9it3s7yl69bjtrnqf,45.203.99.249


# ## 3. DataFrame de Transacciones (`df_transacciones`)
# Contiene variables numéricas de actividad: transacciones, órdenes y pagos.
# **Filtros:**
# 1. Clientes con más de 5 transacciones.
# 2. Clientes con más de 3 órdenes.
# 3. Clientes con pagos = 0.

In [51]:
df_transacciones = df[["customerEmail","No_Transactions","No_Orders","No_Payments"]].copy()
print("DataFrame Transacciones:", df_transacciones.shape)
display(df_transacciones.head(10))

# Filtro 1: más de 5 transacciones
filtro_trans = df_transacciones[df_transacciones["No_Transactions"] > 5]
display(filtro_trans.head())

# Filtro 2: más de 3 órdenes
filtro_ordenes = df_transacciones[df_transacciones["No_Orders"] > 3]
display(filtro_ordenes.head())

# Filtro 3: sin pagos
filtro_sin_pagos = df_transacciones[df_transacciones["No_Payments"] == 0]
display(filtro_sin_pagos.head())

DataFrame Transacciones: (168, 4)


,customerEmail,No_Transactions,No_Orders,No_Payments
0,josephhoward@yahoo.com,2,2,1
1,evansjeffery@yahoo.com,3,3,7
2,andersonwilliam@yahoo.com,5,3,2
3,rubenjuarez@yahoo.com,3,3,1
4,uchen@malone.com,7,7,6
5,robinsoncynthia@dunn.com,1,1,2
6,samuel15@saunders-rhodes.com,2,1,2
7,johnlowery@gmail.com,6,5,2
8,jameslandry@rodriguez.com,5,2,1
9,ubranch@rivera-parker.info,0,0,1


,customerEmail,No_Transactions,No_Orders,No_Payments
4,uchen@malone.com,7,7,6
7,johnlowery@gmail.com,6,5,2
10,bowenwilliam@yahoo.com,6,5,1
11,ksummers@hotmail.com,7,5,4
16,dana09@yahoo.com,6,3,2


,customerEmail,No_Transactions,No_Orders,No_Payments
4,uchen@malone.com,7,7,6
7,johnlowery@gmail.com,6,5,2
10,bowenwilliam@yahoo.com,6,5,1
11,ksummers@hotmail.com,7,5,4
12,christineklein@wright-boyd.org,4,4,3


,customerEmail,No_Transactions,No_Orders,No_Payments
34,zbennett@frazier.com,0,0,0
38,xwang@white.com,0,0,0
44,gsimpson@cox.org,0,0,0
47,johnsonjennifer@yahoo.com,0,0,0
54,zthomas@gmail.com,0,0,0


# ## 4. DataFrame de Riesgo (`df_riesgo`)
# Relaciona clientes con dirección de facturación y la etiqueta de fraude.
# **Filtros:**
# 1. Clientes marcados como fraude = 1.
# 2. Direcciones de facturación duplicadas.
# 3. Clientes con fraude y más de 2 transacciones.

In [52]:
df_riesgo = df[["customerEmail","customerBillingAddress","No_Transactions","Fraud"]].copy()
print("DataFrame Riesgo:", df_riesgo.shape)
display(df_riesgo.head(10))

# Filtro 1: fraude = 1
fraudulentos = df_riesgo[df_riesgo["Fraud"] == 1]
display(fraudulentos.head())

# Filtro 2: direcciones duplicadas
duplicated_address = df_riesgo[df_riesgo.duplicated(subset=["customerBillingAddress"], keep=False)]
display(duplicated_address.head())

# Filtro 3: fraude + más de 2 transacciones
fraude_activo = df_riesgo[(df_riesgo["Fraud"] == 1) & (df_riesgo["No_Transactions"] > 2)]
display(fraude_activo.head())

DataFrame Riesgo: (168, 4)


,customerEmail,customerBillingAddress,No_Transactions,Fraud
0,josephhoward@yahoo.com,"5493 Jones Islands\nBrownside, CA 51896",2,False
1,evansjeffery@yahoo.com,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,3,True
2,andersonwilliam@yahoo.com,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",5,False
3,rubenjuarez@yahoo.com,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",3,False
4,uchen@malone.com,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",7,True
5,robinsoncynthia@dunn.com,"65233 Hutchinson Mountains\nMartinezside, OR 7...",1,True
6,samuel15@saunders-rhodes.com,"3960 Emily Fort\nStaceytown, AR 21817-3325",2,True
7,johnlowery@gmail.com,"484 Pamela Pass\nLake Jessicaview, WI 12942-9074",6,True
8,jameslandry@rodriguez.com,USNS Price\nFPO AE 14661,5,False
9,ubranch@rivera-parker.info,"6974 Robertson Common Apt. 902\nPort Juanside,...",0,True


,customerEmail,customerBillingAddress,No_Transactions,Fraud
1,evansjeffery@yahoo.com,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,3,True
4,uchen@malone.com,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",7,True
5,robinsoncynthia@dunn.com,"65233 Hutchinson Mountains\nMartinezside, OR 7...",1,True
6,samuel15@saunders-rhodes.com,"3960 Emily Fort\nStaceytown, AR 21817-3325",2,True
7,johnlowery@gmail.com,"484 Pamela Pass\nLake Jessicaview, WI 12942-9074",6,True


,customerEmail,customerBillingAddress,No_Transactions,Fraud
57,feliciabrown@gmail.com,49680 Brian Squares Apt. 122\nPort Walterburgh...,1,True
80,warrenedward@arnold.com,49680 Brian Squares Apt. 122\nPort Walterburgh...,4,True
90,brittanydean@hotmail.com,49680 Brian Squares Apt. 122\nPort Walterburgh...,6,True


,customerEmail,customerBillingAddress,No_Transactions,Fraud
1,evansjeffery@yahoo.com,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,3,True
4,uchen@malone.com,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",7,True
7,johnlowery@gmail.com,"484 Pamela Pass\nLake Jessicaview, WI 12942-9074",6,True
10,bowenwilliam@yahoo.com,"27061 Wright Forks\nCraigland, MN 59915",6,True
15,gonzalesjackson@gmail.com,"566 Courtney Stream\nReedland, NH 84853",4,True
